# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
# llm_config = {"model":"gpt-4.1-mini"}

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm_config = {"model":"gemini-1.5-flash", 
            "api_type": "google", 
            "api_key": GOOGLE_API_KEY}


In [2]:
from autogen import ConversableAgent

## Creating the needed agents

In [3]:
onboardingPersonalInformationAgent = ConversableAgent(
    name="onboardingPersonalInformationAgent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
onboardingTopicPreferenceAgent = ConversableAgent(
    name="onboardingTopicPreferenceAgent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [6]:
customerProxyAgent = ConversableAgent(
    name="customerProxyAgent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [7]:
chats = [
    {
        "sender": onboardingPersonalInformationAgent,
        "recipient": customerProxyAgent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                            "into as JSON object only: "
                            "{'name': '', 'location': ''}",
        },
        "max_turns": 4,
        "clear_history" : True
    },
    {
        "sender": onboardingTopicPreferenceAgent,
        "recipient": customerProxyAgent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 2,
        "clear_history" : False
    },
    {
        "sender": customerProxyAgent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 2,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
onboardingPersonalInformationAgent (to customerProxyAgent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/Users/pankajshakya/anaconda3/envs/aiAgentEnv/lib/python3.12/site-packages/autogen/agentchat/chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customerProxyAgent (to onboardingPersonalInformationAgent):

Pankaj , India

--------------------------------------------------------------------------------
onboardingPersonalInformationAgent (to customerProxyAgent):

Thank you Pankaj.  Is there anything else I can help you with today? If not, you're all set.

TERMINATE


--------------------------------------------------------------------------------
customerProxyAgent (to onboardingPersonalInformationAgent):

India

--------------------------------------------------------------------------------
onboardingPersonalInformationAgent (to customerProxyAgent):

Thank you. I have your location.  Is there anything else I can help you with today? If not, you're all set.

TERMINATE


--------------------------------------------------------------------------------
customerProxyAgent (to onboardingPersonalInformationAgent):

Indian

--------------------------------------------------------------------------------
onboardingPersonalInformationAge

## Print out the summary

In [11]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'content': '```json\n{"name": "Pankaj", "location": "India"}\n```\n', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'The conversation established two areas of interest:  a person named Rohit, and AI agents in finance.\n', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'Pankaj, from India, showed interest in Rohit and AI in finance.  A fictional story about Rohit, a clumsy AI agent in a Mumbai bank, was suggested as a fun reading option.\n', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}




## Print out the cost

In [12]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 7.365e-05, 'gemini-1.5-flash': {'cost': 7.365e-05, 'prompt_tokens': 518, 'completion_tokens': 116, 'total_tokens': 634}}, 'usage_excluding_cached_inference': {'total_cost': 7.365e-05, 'gemini-1.5-flash': {'cost': 7.365e-05, 'prompt_tokens': 518, 'completion_tokens': 116, 'total_tokens': 634}}}


{'usage_including_cached_inference': {'total_cost': 2.3775e-05, 'gemini-1.5-flash': {'cost': 2.3775e-05, 'prompt_tokens': 177, 'completion_tokens': 35, 'total_tokens': 212}}, 'usage_excluding_cached_inference': {'total_cost': 2.3775e-05, 'gemini-1.5-flash': {'cost': 2.3775e-05, 'prompt_tokens': 177, 'completion_tokens': 35, 'total_tokens': 212}}}


{'usage_including_cached_inference': {'total_cost': 0.000164925, 'gemini-1.5-flash': {'cost': 0.000164925, 'prompt_tokens': 815, 'completion_tokens': 346, 'total_tokens': 1161}}, 'usage_excluding_cached_inference': {'total_cost': 0.000164925, 'gemini-1.5-flash': {'cost': 0.000164925, 'prompt_tokens'